In [1]:
from wordcloud import WordCloud
import pandas as pd
from nltk.tokenize import word_tokenize 
import matplotlib.pyplot as plt
#Find the ten most/least common words
from collections import Counter

In [2]:
df = pd.read_csv("../data/Preprocessed_Newsarticles_1okt2022_V2.csv")
df_apple = df[df["ticker"]=="AAPL" ]
df_apple["target"] = (df_apple["close"] > df_apple["open"]).astype(int)
df_apple.head(5)


C:\Users\groot\AppData\Local\Temp\ipykernel_876\1160405396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_apple["target"] = (df_apple["close"] > df_apple["open"]).astype(int)


,Unnamed: 0,id,ticker,title,category,content,date,provider,url,open,high,low,close,adj_close,volume,target
8491,8491,290924,AAPL,Waiting For Direction On The Markets,opinion,stock market difficult trader investor alike d...,2012-07-16,Cam Hui,https://www.investing.com/analysis/waiting-for...,21.611429,21.843571,21.607857,21.675358,18.507113,301260400,1
8546,8546,290925,AAPL,Mid Year Update U S And Canadian Stock Marke...,opinion,2004 2010 tsx 300 index lead canadian stock ou...,2012-07-19,Baskin Financial Blog,https://www.investing.com/analysis/mid-year-up...,21.831429,21.976786,21.642857,21.940001,18.733074,436861600,1
8591,8591,290928,AAPL,Trade Apple After Earnings,opinion,look like spider web mishmash trendline make i...,2012-07-23,Abigail Doolittle,https://www.investing.com/analysis/trade-apple...,21.228571,21.639286,20.989643,21.565357,18.413191,487975600,1
8592,8592,290927,AAPL,Apple Earnings Preview Quarterly Dip On Deck,opinion,quarter apple AAPL report 2nd good quarter com...,2012-07-23,David Dyer,https://www.investing.com/analysis/apple-earni...,21.228571,21.639286,20.989643,21.565357,18.413191,487975600,1
8593,8593,290926,AAPL,Summer Heat Scorches Europe And U S,opinion,europe flare summer heat continue summer heat ...,2012-07-23,John Nyaradi,https://www.investing.com/analysis/summer-heat...,21.228571,21.639286,20.989643,21.565357,18.413191,487975600,1


In [3]:
df_apple_increase = df_apple[df_apple['target']==1]
tokens_pos = df_apple_increase['content'].apply(word_tokenize)

ALL_tokens_positive = []
for token_list in tokens_pos:
    ALL_tokens_positive.extend(token_list)

    
df_apple_decrease = df_apple[df_apple['target']==0]
tokens_neg = df_apple_decrease['content'].apply(word_tokenize)    

ALL_tokens_neg = []
for token_list in tokens_neg:
    ALL_tokens_neg.extend(token_list)
    

In [4]:
#Find the ten most/least common words
from collections import Counter
counted_pos = Counter(ALL_tokens_positive)
counted_neg = Counter(ALL_tokens_neg)

In [62]:
Intersect = (counted_pos - counted_neg)

for item, count in Intersect.items():

    Intersect[item] /= counted_pos[item] + counted_neg[item]

intersect_above_80 = Counter({x: count for x, count in Intersect.items() if count >= 0.8})

Pos_candidates = []
for item, count in intersect_above_80.items():
    if counted_pos[item] > 10:
        Pos_candidates.append((item,counted_pos[item]))


In [63]:
print(Pos_candidates)

[('rimm', 23), ('772', 11), ('893', 14), ('rga', 15), ('mattress', 11), ('blah', 12), ('towson', 11), ('suri', 11), ('checklist', 16), ('iv30', 11), ('hv180', 14), ('hv360', 12), ('ww', 31), ('jnk', 12), ('bombardi', 15), ('sp500', 15), ('cx', 15), ('tinder', 22), ('isle', 19), ('conn', 29), ('islands', 11), ('vinci', 22), ('branded', 16), ('mann', 17), ('mcx', 29), ('medallion', 11), ('nxt', 27), ('wocket', 13), ('sri', 31), ('paulson', 11), ('deepa', 11), ('seetharaman', 11), ('itv', 32), ('mont', 13), ('celldex', 19), ('nude', 15), ('140k', 11), ('vinh', 14), ('vail', 15), ('tesoro', 23), ('sovaldi', 19), ('harvoni', 14), ('parts', 13), ('misfit', 15), ('onedrive', 19), ('kwh', 12), ('screenshot', 12), ('conciliatory', 12), ('1198', 11), ('1165', 20), ('dyson', 30), ('cornell', 22), ('schmidt', 32), ('contrafund', 16), ('conde', 11), ('computex', 13), ('hercules', 12), ('saw', 12), ('vestberg', 13), ('ase', 18), ('korn', 11), ('657', 16), ('megan', 13), ('albrecht', 13), ('harbour',

In [ ]:
#Initialize word cloud
moby_dick_str = " ".join(ALL_tokens_positive)
wordcloud = WordCloud(width = 1600, height = 800,
background_color ='white',
min_font_size =
10).generate(moby_dick_str)

#Create the Word cloud
plt.figure(figsize = (16, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
#Initialize word cloud
moby_dick_str = " ".join(ALL_tokens_neg)
wordcloud = WordCloud(width = 1600, height = 800,
background_color ='white',
min_font_size =
10).generate(moby_dick_str)

#Create the Word cloud
plt.figure(figsize = (16, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [31]:
from nltk.util import ngrams

In [33]:
def extract_ngrams(data, num):
    n_grams = ngrams(data, num)
    return [ ' '.join(grams) for grams in n_grams]

In [35]:
x = extract_ngrams(ALL_tokens_positive,2)
Ngrams_pos_counted = Counter(x)

In [36]:
Ngrams_pos_counted.most_common(50)

[('u s', 11756),
 ('zack rank', 9433),
 ('nasdaq AAPL', 7938),
 ('s p', 7157),
 ('company s', 6535),
 ('p 500', 6106),
 ('apple nasdaq', 6091),
 ('consensus estimate', 5694),
 ('year year', 5004),
 ('zack consensus', 4473),
 ('inc nasdaq', 4344),
 ('don t', 3658),
 ('apple s', 3470),
 ('10 year', 3404),
 ('long term', 3182),
 ('1 billion', 3163),
 ('special report', 3042),
 ('inc nyse', 2952),
 ('3 year', 2931),
 ('stock news', 2810),
 ('1 7', 2809),
 ('year ago', 2788),
 ('rank 1', 2679),
 ('rank 2', 2632),
 ('apple inc', 2618),
 ('zack release', 2599),
 ('rank 3', 2569),
 ('release special', 2535),
 ('united states', 2436),
 ('carry zack', 2434),
 ('expect generate', 2417),
 ('take advantage', 2413),
 ('trillion market', 2396),
 ('27 billion', 2395),
 ('year new', 2393),
 ('7 trillion', 2383),
 ('t buy', 2382),
 ('billion iphone', 2371),
 ('market zack', 2362),
 ('news big', 2350),
 ('billion device', 2350),
 ('report spotlight', 2349),
 ('create 1', 2349),
 ('year create', 2348),
 (

In [37]:
x = extract_ngrams(ALL_tokens_neg,2)
Ngrams_neg_counted = Counter(x)
Ngrams_neg_counted.most_common(50)

[('u s', 10303),
 ('zack rank', 7280),
 ('nasdaq AAPL', 6610),
 ('s p', 6187),
 ('p 500', 5340),
 ('company s', 5271),
 ('apple nasdaq', 5021),
 ('consensus estimate', 3962),
 ('inc nasdaq', 3485),
 ('year year', 3472),
 ('apple s', 3173),
 ('zack consensus', 3122),
 ('don t', 3060),
 ('10 year', 2735),
 ('long term', 2652),
 ('1 billion', 2492),
 ('inc nyse', 2336),
 ('apple inc', 2299),
 ('3 year', 2218),
 ('special report', 2215),
 ('rank 1', 2142),
 ('1 7', 2112),
 ('year ago', 2086),
 ('rank 3', 2079),
 ('stock news', 2041),
 ('united states', 2025),
 ('zack release', 2011),
 ('release special', 1947),
 ('year new', 1867),
 ('take advantage', 1864),
 ('expect generate', 1860),
 ('trillion market', 1860),
 ('27 billion', 1859),
 ('earning growth', 1855),
 ('t buy', 1853),
 ('7 trillion', 1841),
 ('rank 2', 1838),
 ('market zack', 1832),
 ('billion iphone', 1829),
 ('news big', 1819),
 ('billion device', 1819),
 ('mere 1', 1816),
 ('iphone 10', 1816),
 ('big iphone', 1815),
 ('fast 